In [ ]:
import os
import io
import sys

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

from youtube_transcript_api import YouTubeTranscriptApi

scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [5]:
# method to gather the channel id from channel url
# param: url - string
# return: channel id - string
# raise ValueError when input is not a valid url
def get_channel_id_from_url(url):
    if not url.startswith("https://www.youtube.com/channel/"):
        raise ValueError("input url not a url")
    else:
        return url[len("https://www.youtube.com/channel/"):]

# print(get_channel_id_from_url("https://www.youtube.com/channel/UCbAwSkqJ1W_Eg7wr3cp5BUA"))

UCbAwSkqJ1W_Eg7wr3cp5BUA


In [ ]:
# method to get video ids from channel id
def get_vidids_from_channel(channel_id):
    

In [ ]:
# method to get transcript from video ids
# params: video_ids - a list of video ids
# TODO return type?
def get_transcript_from_vidids(video_ids):
    script = []
    for i in range(len(video_ids)):
        script.append(YouTubeTranscriptApi.get_transcript(video_ids[i]))
    print(script)

    with open('sample_caption.txt',"w") as filehandle:
        for i in range(len(script)):
            for listitem in script[i]:
                filehandle.write(listitem.get('text')+" ")


In [ ]:
def main():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "0"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "client_secret.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)
